In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
import seaborn as sns
import plotly.express as px
import json
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from torch.utils.data import TensorDataset,DataLoader
from tqdm import tqdm

# Train Model

In [ ]:
# Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
# Define Model
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(300, 10),
            nn.ReLU(),
            nn.Linear(10, 1)
        )
    
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits  

model = MLP().to(device)
    
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# Define training parameters and preapre dataloaders from saved datasets

epochs = 10
batch_size = 64

losses = []
test_losses = []

train_dataset = torch.load('pipeline/datasets/train_dataset.pt')
test_dataset = torch.load('pipeline/datasets/test_dataset.pt')

train_length = len(train_dataset)
test_length = len(test_dataset)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset)

In [ ]:
# Train

for epoch in range(epochs):
    for X_train_batch, y_train_batch in train_dataloader:
        
        X_train_batch = X_train_batch.to(device)
        y_train_batch = y_train_batch.to(device)
        
        # Forward Pass
        logits = model(X_train_batch)

        loss = criterion(logits, y_train_batch)

        # Backward Pass
        optimizer.zero_grad
        loss.backward()
        optimizer.step()

        losses.append(loss.cpu().item())
    print(f'Epoch {epoch} - loss: {losses[-train_length:]/train_length}')

In [ ]:
# Plot loss curve at specified resolution

res = 100
plt.plot(torch.tensor(losses)[:len(losses)-len(losses)%res].view(-1,res).mean(1))

In [ ]:
# Save model

torch.save

# Test

In [ ]:
# Test by Predicting with Model

# Get one file
index = 53
X_pred = torch.from_numpy(np.load(f'pipeline/2_windowed/{index}.npy')).float()
y_true = torch.from_numpy(np.load(f'pipeline/labels/{index}.npy')).reshape(-1,1).float()

model.eval()

step = 10000    # for memory
preds = []
correct = 0
loss = 0
for i in range(0, len(X_pred), step):
    end = i+step if i+step < len(X_pred) else len(X_pred)

    X_batch = X_pred[i:end].to(device)    
    y_true_batch = y_true[i:end].to(device)

    logits = model(X_batch)
    pred = torch.round(nn.Sigmoid()(logits))
    correct += sum(y_true_batch == pred)
    preds += pred.flatten().tolist()
    loss += loss_fn(logits, y_true_batch).item()

preds = np.array(preds).reshape(-1,1)
accuracy = (correct / len(y_true)).item()
loss = loss/len(X_pred)
print(f'Accuracy: {100*accuracy:.4}%')
print(f'Loss: {loss:.4}')

In [ ]:
# Visualize predictions
df_pred = pd.read_csv(f'pipeline/1_xyz/{index}.csv')
df_pred['preds'] = np.pad(preds.flatten()*10, (0, 99), mode='constant', constant_values=0)

figure = px.line(df_pred.loc[::10])
figure.show(renderer='browser')

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true.detach().to('cpu'), preds)
sns.heatmap(cm, annot=True)